# Технологии обработки больших данных

Занятие 4. Работа с SQL запросами в PySpark 

1. Запуск PySpark на локальной машине
2. TempView - представления DataFrame
3. Базовый синтаксис SQL
4. Домашнее задание 
  
**Рекомендованная литература:** Б.Форта. SQL за 10 минут.

![](img/spark_libs.png)

### 1. Запуск PySpark на локальной машине

! pip install pyspark

In [1]:
import pyspark

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

22/03/21 23:05:07 WARN Utils: Your hostname, calc resolves to a loopback address: 127.0.1.1; using 192.168.122.1 instead (on interface virbr0)
22/03/21 23:05:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/21 23:05:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### 2. View - представления DataFrame

**Columns**  

Age - возраст (numeric)  
Sex - пол (text: male, female)  
Job - уровень рабочей квалификации (numeric: 0 - unskilled and non-resident, 1 - unskilled and resident, 2 - skilled, 3 - highly skilled)  
Housing - жилье (text: own, rent, or free)  
Saving accounts - накопления (text - little, moderate, quite rich, rich)  
Checking account - средние траты (numeric, in DM - Deutsch Mark)  
Credit amount - размер кредита (numeric, in DM)  
Duration - срок кредита (numeric, in month)  
Purpose - цель кредита (text: car, furniture/equipment, radio/TV, domestic appliances, repairs, education, business, vacation/others)

In [2]:
DATA_PATH = 'sample_data/credit_data.csv'

SCHEMA = "id INT, Age INT, Sex STRING, Job INT, Housing STRING, Saving_accounts STRING, \
Checking_account STRING, Credit_amount INT, Duration INT, Purpose STRING"

df = spark.read.csv(DATA_PATH, schema=SCHEMA, header=True)
type(df)

pyspark.sql.dataframe.DataFrame

In [3]:
df.createOrReplaceTempView("credit_table")

С помощью запросов SQL можно создавать новые представления DataFrame

In [4]:
sub_df = spark.sql("SELECT id, Age, Sex FROM credit_table ")
sub_df.show(10)

+---+---+------+
| id|Age|   Sex|
+---+---+------+
|  0| 67|  male|
|  1| 22|female|
|  2| 49|  male|
|  3| 45|  male|
|  4| 53|  male|
|  5| 35|  male|
|  6| 53|  male|
|  7| 35|  male|
|  8| 61|  male|
|  9| 28|  male|
+---+---+------+
only showing top 10 rows



### 3. Базовый синтаксис SQL

#### Извлечение отдельных столбцов

In [5]:
spark.sql("SELECT Age FROM credit_table ").show(5)

+---+
|Age|
+---+
| 67|
| 22|
| 49|
| 45|
| 53|
+---+
only showing top 5 rows



#### Извлечение всех столбцов

In [6]:
spark.sql("SELECT * FROM credit_table ").show(5)

+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+
| id|Age|   Sex|Job|Housing|Saving_accounts|Checking_account|Credit_amount|Duration|            Purpose|
+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+
|  0| 67|  male|  2|    own|             NA|          little|         1169|       6|           radio/TV|
|  1| 22|female|  2|    own|         little|        moderate|         5951|      48|           radio/TV|
|  2| 49|  male|  1|    own|         little|              NA|         2096|      12|          education|
|  3| 45|  male|  2|   free|         little|          little|         7882|      42|furniture/equipment|
|  4| 53|  male|  2|   free|         little|          little|         4870|      24|                car|
+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+
only showing top 5 rows



#### Извлечение уникальных строк

In [7]:
spark.sql("SELECT DISTINCT Purpose FROM credit_table ").show()

+-------------------+
|            Purpose|
+-------------------+
|          education|
|           radio/TV|
|    vacation/others|
|domestic appliances|
|                car|
|           business|
|            repairs|
|furniture/equipment|
+-------------------+



#### Ограничение результатов запроса

In [8]:
spark.sql("SELECT * FROM credit_table LIMIT 5").show()

+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+
| id|Age|   Sex|Job|Housing|Saving_accounts|Checking_account|Credit_amount|Duration|            Purpose|
+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+
|  0| 67|  male|  2|    own|             NA|          little|         1169|       6|           radio/TV|
|  1| 22|female|  2|    own|         little|        moderate|         5951|      48|           radio/TV|
|  2| 49|  male|  1|    own|         little|              NA|         2096|      12|          education|
|  3| 45|  male|  2|   free|         little|          little|         7882|      42|furniture/equipment|
|  4| 53|  male|  2|   free|         little|          little|         4870|      24|                car|
+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+



#### Сортировка записей

In [9]:
spark.sql("SELECT * FROM credit_table ORDER BY Duration").show(10)

+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+
| id|Age|   Sex|Job|Housing|Saving_accounts|Checking_account|Credit_amount|Duration|            Purpose|
+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+
|234| 42|  male|  1|    own|         little|              NA|         1544|       4|           radio/TV|
|943| 42|  male|  1|    own|         little|              NA|         1455|       4|                car|
|527| 42|  male|  1|    own|         little|              NA|         1503|       4|           radio/TV|
|734| 37|female|  2|    own|         little|              NA|         3380|       4|                car|
|314| 29|  male|  1|    own|             NA|            rich|         1494|       4|                car|
|837| 23|female|  1|   rent|         little|              NA|          601|       4|furniture/equipment|
|430| 74|  male|  1|    own|         little|           

In [10]:
# Few columns sorting
spark.sql("SELECT * FROM credit_table ORDER BY Duration, Credit_amount").show(10)

+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+
| id|Age|   Sex|Job|Housing|Saving_accounts|Checking_account|Credit_amount|Duration|            Purpose|
+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+
|837| 23|female|  1|   rent|         little|              NA|          601|       4|furniture/equipment|
|943| 42|  male|  1|    own|         little|              NA|         1455|       4|                car|
|314| 29|  male|  1|    own|             NA|            rich|         1494|       4|                car|
|527| 42|  male|  1|    own|         little|              NA|         1503|       4|           radio/TV|
|234| 42|  male|  1|    own|         little|              NA|         1544|       4|           radio/TV|
|734| 37|female|  2|    own|         little|              NA|         3380|       4|                car|
|430| 74|  male|  1|    own|         little|           

#### Сортировка по положению столбца

In [11]:
spark.sql("SELECT * FROM credit_table ORDER BY 2, 3").show()

+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+
| id|Age|   Sex|Job|Housing|Saving_accounts|Checking_account|Credit_amount|Duration|            Purpose|
+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+
|391| 19|female|  1|   rent|           rich|        moderate|          983|      12|furniture/equipment|
|633| 19|female|  2|   rent|         little|              NA|         1980|       9|furniture/equipment|
|155| 20|female|  2|   rent|         little|          little|         1282|      12|furniture/equipment|
|296| 20|female|  2|   rent|             NA|              NA|         4675|      12|                car|
|410| 20|female|  2|    own|         little|        moderate|         1967|      24|           radio/TV|
|420| 20|female|  2|   rent|           rich|              NA|         3186|      15|                car|
|585| 20|female|  2|   rent|         little|          l

####  Направления сортировки

In [12]:
spark.sql("SELECT * FROM credit_table ORDER BY Duration DESC").show(10)

+---+---+------+---+-------+---------------+----------------+-------------+--------+---------------+
| id|Age|   Sex|Job|Housing|Saving_accounts|Checking_account|Credit_amount|Duration|        Purpose|
+---+---+------+---+-------+---------------+----------------+-------------+--------+---------------+
|677| 24|  male|  2|    own|       moderate|        moderate|         5595|      72|       radio/TV|
|938| 42|  male|  2|   free|         little|        moderate|         6288|      60|      education|
|255| 27|  male|  1|    own|             NA|        moderate|         7418|      60|       radio/TV|
|332| 24|female|  3|    own|       moderate|        moderate|         7408|      60|            car|
|672| 42|  male|  3|    own|         little|              NA|        10366|      60|            car|
|373| 63|  male|  3|   free|             NA|              NA|        13756|      60|            car|
|134| 21|female|  2|    own|       moderate|              NA|        10144|      60|       

In [13]:
spark.sql("SELECT * FROM credit_table ORDER BY Credit_amount DESC, Duration ").show(10)

+---+---+------+---+-------+---------------+----------------+-------------+--------+---------------+
| id|Age|   Sex|Job|Housing|Saving_accounts|Checking_account|Credit_amount|Duration|        Purpose|
+---+---+------+---+-------+---------------+----------------+-------------+--------+---------------+
|915| 32|female|  3|    own|         little|        moderate|        18424|      48|vacation/others|
| 95| 58|  male|  2|   rent|         little|        moderate|        15945|      54|       business|
|818| 43|  male|  3|    own|         little|          little|        15857|      36|vacation/others|
|887| 23|  male|  2|    own|         little|        moderate|        15672|      48|       business|
|637| 21|  male|  2|    own|         little|              NA|        15653|      60|       radio/TV|
|917| 68|  male|  3|    own|         little|          little|        14896|       6|            car|
|374| 60|female|  3|   free|       moderate|        moderate|        14782|      60|vacatio

#### Фильтрация данных WHERE

In [16]:
spark.sql("SELECT * FROM credit_table WHERE Purpose = 'car' ").show(10)

+---+---+------+---+-------+---------------+----------------+-------------+--------+-------+
| id|Age|   Sex|Job|Housing|Saving_accounts|Checking_account|Credit_amount|Duration|Purpose|
+---+---+------+---+-------+---------------+----------------+-------------+--------+-------+
|  4| 53|  male|  2|   free|         little|          little|         4870|      24|    car|
|  7| 35|  male|  3|   rent|         little|        moderate|         6948|      36|    car|
|  9| 28|  male|  3|    own|         little|        moderate|         5234|      30|    car|
| 10| 25|female|  2|   rent|         little|        moderate|         1295|      12|    car|
| 13| 60|  male|  1|    own|         little|          little|         1199|      24|    car|
| 14| 28|female|  2|   rent|         little|          little|         1403|      15|    car|
| 18| 44|female|  3|   free|         little|        moderate|        12579|      24|    car|
| 20| 48|  male|  2|    own|         little|              NA|         

#### Операторы WHERE  

In [18]:
#  =  Равенство
#  <> Неравенство
#  != Неравенство
#  <  Меньше
#  <= Меньше или равно
#  !< Не меньше
#  >  Больше
#  >= Больше или равно
#  !> Не больше
#  BETWEEN Вхождение в диапазон
#  IS NULL Значение NULL

In [19]:
spark.sql("SELECT * FROM credit_table WHERE Housing != 'free' ").show(10)

+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+
| id|Age|   Sex|Job|Housing|Saving_accounts|Checking_account|Credit_amount|Duration|            Purpose|
+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+
|  0| 67|  male|  2|    own|             NA|          little|         1169|       6|           radio/TV|
|  1| 22|female|  2|    own|         little|        moderate|         5951|      48|           radio/TV|
|  2| 49|  male|  1|    own|         little|              NA|         2096|      12|          education|
|  6| 53|  male|  2|    own|     quite rich|              NA|         2835|      24|furniture/equipment|
|  7| 35|  male|  3|   rent|         little|        moderate|         6948|      36|                car|
|  8| 61|  male|  1|    own|           rich|              NA|         3059|      12|           radio/TV|
|  9| 28|  male|  3|    own|         little|        mod

In [20]:
spark.sql("SELECT * FROM credit_table WHERE Credit_amount BETWEEN 1000 AND 2000 ").show(10)

+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+
| id|Age|   Sex|Job|Housing|Saving_accounts|Checking_account|Credit_amount|Duration|            Purpose|
+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+
|  0| 67|  male|  2|    own|             NA|          little|         1169|       6|           radio/TV|
| 10| 25|female|  2|   rent|         little|        moderate|         1295|      12|                car|
| 12| 22|female|  2|    own|         little|        moderate|         1567|      12|           radio/TV|
| 13| 60|  male|  1|    own|         little|          little|         1199|      24|                car|
| 14| 28|female|  2|   rent|         little|          little|         1403|      15|                car|
| 15| 32|female|  1|    own|       moderate|          little|         1282|      24|           radio/TV|
| 23| 44|  male|  2|    own|       moderate|        mod

#### Комбинирование условий WHERE

In [27]:
spark.sql("""SELECT * FROM credit_table 
                WHERE Credit_amount > 10000
                  AND Purpose = 'car' 
                ORDER BY Credit_amount
                """).show(10)

+---+---+------+---+-------+---------------+----------------+-------------+--------+-------+
| id|Age|   Sex|Job|Housing|Saving_accounts|Checking_account|Credit_amount|Duration|Purpose|
+---+---+------+---+-------+---------------+----------------+-------------+--------+-------+
|304| 44|  male|  2|   free|     quite rich|              NA|        10127|      48|    car|
|927| 39|  male|  2|   free|         little|          little|        10297|      48|    car|
|672| 42|  male|  3|    own|         little|              NA|        10366|      60|    car|
|902| 42|  male|  2|   free|             NA|              NA|        10477|      36|    car|
|205| 38|  male|  3|   free|         little|          little|        10623|      30|    car|
|285| 35|female|  1|    own|         little|          little|        10722|      47|    car|
|854| 45|  male|  2|    own|         little|              NA|        10875|      36|    car|
|450| 30|  male|  3|    own|             NA|              NA|        1

#### Условие OR

In [28]:
spark.sql("""SELECT * FROM credit_table 
                WHERE Purpose = 'furniture/equipment'
                   OR Purpose = 'business' 
                ORDER BY Credit_amount
                """).show(10)

+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+
| id|Age|   Sex|Job|Housing|Saving_accounts|Checking_account|Credit_amount|Duration|            Purpose|
+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+
|750| 49|female|  2|    own|         little|          little|          428|       6|furniture/equipment|
|837| 23|female|  1|   rent|         little|              NA|          601|       4|furniture/equipment|
|439| 26|female|  0|    own|         little|            rich|          609|      12|           business|
|869| 24|female|  2|   rent|         little|          little|          652|      12|furniture/equipment|
|316| 38|  male|  1|    own|         little|          little|          708|      12|furniture/equipment|
|171| 26|female|  2|    own|         little|              NA|          763|      12|furniture/equipment|
|573| 22|female|  1|    own|         little|          l

####  Условие IN

In [29]:
spark.sql("""SELECT * FROM credit_table 
                WHERE Purpose IN ('furniture/equipment', 'business')
                ORDER BY Credit_amount
                """).show(10)

+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+
| id|Age|   Sex|Job|Housing|Saving_accounts|Checking_account|Credit_amount|Duration|            Purpose|
+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+
|750| 49|female|  2|    own|         little|          little|          428|       6|furniture/equipment|
|837| 23|female|  1|   rent|         little|              NA|          601|       4|furniture/equipment|
|439| 26|female|  0|    own|         little|            rich|          609|      12|           business|
|869| 24|female|  2|   rent|         little|          little|          652|      12|furniture/equipment|
|316| 38|  male|  1|    own|         little|          little|          708|      12|furniture/equipment|
|171| 26|female|  2|    own|         little|              NA|          763|      12|furniture/equipment|
|573| 22|female|  1|    own|         little|          l

####  Условие NOT

In [32]:
spark.sql("""SELECT * FROM credit_table 
                WHERE Purpose IN ('furniture/equipment', 'business')
                  AND NOT Job = 0
                ORDER BY Credit_amount
                """).show(10)

+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+
| id|Age|   Sex|Job|Housing|Saving_accounts|Checking_account|Credit_amount|Duration|            Purpose|
+---+---+------+---+-------+---------------+----------------+-------------+--------+-------------------+
|750| 49|female|  2|    own|         little|          little|          428|       6|furniture/equipment|
|837| 23|female|  1|   rent|         little|              NA|          601|       4|furniture/equipment|
|869| 24|female|  2|   rent|         little|          little|          652|      12|furniture/equipment|
|316| 38|  male|  1|    own|         little|          little|          708|      12|furniture/equipment|
|171| 26|female|  2|    own|         little|              NA|          763|      12|furniture/equipment|
|573| 22|female|  1|    own|         little|          little|          806|      15|           business|
|752| 23|female|  1|   rent|       moderate|        mod